In [1]:
%cd ../

D:\WorkFolder\nlp_thes_augm


In [2]:
from nlp_thes_augm.utils.wordnet import RuWordNet
from nlp_thes_augm.models.augmentation import WordnetAugmentator

wordnet_path = r'D:\WorkFolder\data\models\RuWordNet'
wordnet = RuWordNet(wordnet_path)

t5_model_path = r'D:\WorkFolder\data\models\rut5_base_restorer_1m'
gpt2_model_path = 'sberbank-ai/rugpt3small_based_on_gpt2'
wordnet_augmentator = WordnetAugmentator(wordnet, t5_model_path, gpt2_model_path)

C:\ProgramData\Anaconda\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [3]:
text = 'В Совфеде при президент рф обсудили важнейший проект по освоению российской Арктики, президент рф согласен с позицией.'
augmentations, _ = wordnet_augmentator.augmentate(
    text, augmentations_count=100, topk=5, max_concept_count=5, bs=100
)
for augm in augmentations:
    print(augm.restored_augmentation)
    print(augm.score)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 116.55it/s]

В Совфеде при президенте России пообсуждали важнейший проект по освоению российской Арктики, президент Российской Федерации согласен с позицией.
45.1832389831543
В Совфеде при президенте РФ обговорили важнейшую реализацию проекта по освоению российской Арктики, глава российского государства согласен с позицией.
51.7611083984375
В Совфеде при президенте РФ обсудили важнейший проект по освоению российской Арктики, президент РФ согласен с позицией воинской части.
52.104129791259766
В Совфеде при президенте России обсудили важный проект по освоению общероссийской Арктики, президент РФ согласен с положениями.
53.42341232299805
В Совфеде при президенте России пообсуждали важнейший проект по освоению российского Арктического района, президент РФ согласен с положениями.
53.84498596191406


In [ ]:
augm.restored_augmentation

In [ ]:
augm.senses

In [ ]:
augm.concepts[0].tokens

In [ ]:
augm.restored_augmentation

In [ ]:
wordnet_augmentator.concept_detector.detect(augm.restored_augmentation)[0].get_sense()

In [7]:
import re
augm_text_tokenized = re.findall(wordnet_augmentator.text_preprocessor.sep_reg, augm.restored_augmentation)
augm_concepts = augm.concepts
augm_senses = augm.senses
augm_synsets_ids = augm.synsets_ids

In [19]:
def markup_augmentation(wordnet_augmentator, wordnet, augm):
    augm_text_tokenized = re.findall(wordnet_augmentator.text_preprocessor.sep_reg, augm.restored_augmentation)
    augm_concepts = augm.concepts
    augm_senses = augm.senses
    augm_synsets_ids = augm.synsets_ids

    current_concept_idx = 0
    current_concept_num = augm_concepts[current_concept_idx].tokens[0].num
    shift = 0
    counter = 0

    result_markup_text = ''
    for i, token in enumerate(augm_text_tokenized):
        if counter > 0:
            result_markup_text += ' ' + token
            if counter == 1:
                result_markup_text += '</u>'
            counter -= 1
            continue
        if i == current_concept_num + shift:

            selected_synset_name = wordnet.synsets[augm_synsets_ids[current_concept_idx]].synset_name
            selected_sense =  augm_senses[current_concept_idx]

            hover_text = f'<b>Выбранный концепт:</b><br>{selected_synset_name}<br><b>Выбранный текстовый вход:</b><br>{selected_sense}'
            open_tag = f'<u data-toggle=\"tooltip\" data-html=\"true\" title=\"{hover_text}\">'
            result_markup_text += open_tag

            counter = len(augm_senses[current_concept_idx].split('_'))
            shift += counter - len(augm_concepts[current_concept_idx].tokens)
            current_concept_idx += 1
            current_concept_num = augm_concepts[current_concept_idx].tokens[0].num if current_concept_idx < len(augm_concepts) else len(augm_text_tokenized)

        result_markup_text += ' ' + token
        if counter == 1:
            result_markup_text += '</u>'
        counter -= 1
    return result_markup_text.strip()

In [21]:
markup_augmentation(wordnet_augmentator, wordnet, augmentations[1])

'В Совфеде при президенте РФ<u data-toggle="tooltip" data-html="true" title="<b>Выбранный концепт:</b><br>обсуждение<br><b>Выбранный текстовый вход:</b><br>обговорить"> обговорили</u> важнейшую<u data-toggle="tooltip" data-html="true" title="<b>Выбранный концепт:</b><br>проект (комплекс взаимосвязанных работ)<br><b>Выбранный текстовый вход:</b><br>реализация_проект"> реализацию проекта</u> по освоению российской<u data-toggle="tooltip" data-html="true" title="<b>Выбранный концепт:</b><br>арктика<br><b>Выбранный текстовый вход:</b><br>арктика"> Арктики</u> ,<u data-toggle="tooltip" data-html="true" title="<b>Выбранный концепт:</b><br>президент россии<br><b>Выбранный текстовый вход:</b><br>глава_российский_государство"> глава российского государства</u><u data-toggle="tooltip" data-html="true" title="<b>Выбранный концепт:</b><br>согласный звук<br><b>Выбранный текстовый вход:</b><br>согласный"> согласен</u> с позицией .'

In [ ]:
dir(augm_concepts[0].synsets_ids)

In [ ]:
augm_concepts = augm.concepts
augm_senses = augm.senses

In [ ]:
augm.concepts[0].tokens[0].num, len(augm.concepts[0].tokens)

In [ ]:
len(augm.senses[0].split('_'))

In [ ]:
wordnet.sense2synid['президент_рф']

In [ ]:
augm.restored_augmentation

In [ ]:
dir(augm)

In [ ]:
from nlp_thes_augm.utils.common import inject_concept_tokens
inject_concept_tokens(augm.concepts, augm.text)

In [ ]:
import string
string.punctuation

In [ ]:
import pymorphy2
from functools import lru_cache
import string 
import re
import numpy as np
import copy
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm import tqdm


class Token:
    def __init__(self, word, token, num, shift):
        self.word = word
        self.word_len = len(word)
        self.token = token
        self.num = num
        self.shift = shift
        
class ConceptToken:
    def __init__(self, tokens, synsets_ids):
        self.tokens = tokens
        self.synsets_ids = synsets_ids
        
    def get_sense(self):
        return ' '.join([t.word for t in self.tokens])
    
    def get_sense_norm(self):
        return '_'.join([t.token for t in self.tokens])
        
    def __str__(self):
        sense = self.get_sense()
        sense_norm = self.get_sense_norm()
        synsets = ','.join(self.synsets_ids)
        return '{{' + sense + '|' + sense_norm + '|' + synsets + '}}'
        
class TextPreprocessor:
    def __init__(self):
        self.sep_reg = re.compile(r"[\w]+|[!\"#$%&\'()*+,-–—./:;<=>?@\[\\\]^_`{|}~“”«»]")
        self.morph_analyzer = pymorphy2.MorphAnalyzer()
    
    @lru_cache(maxsize=1000000)
    def get_normal_form(self, word):
        return self.morph_analyzer.parse(word)[0].normal_form
    
    def tokenize_text(self, text):
        text_splitted = re.findall(self.sep_reg, text)
        shift = 0
        result = []
        for i, w in enumerate(text_splitted):
            w_norm = self._normalize_word(w)
            if text[shift] == ' ':
                shift += 1
            w_shift = shift
            shift = shift + len(w)
            result.append(Token(w, w_norm, i, w_shift))

        return result
    
    def _normalize_word(self, word):
        if (not word.isalpha()) or (len(word) < 3) or word.isupper():
            return word

        return self.get_normal_form(word).lower()

class ConceptDetector:
    def __init__(self, wordnet, text_preprocessor):
        self.wordnet = wordnet
        self.senses = {}
        self.issensecontinuation = set()
        self.text_preprocessor = text_preprocessor
        self._transform_senses()

    def _transform_senses(self):
        for sense in self.wordnet.senses:
            splited_sense = sense.split('_')
            sense_len = len(splited_sense)
            for i in range(1, sense_len):
                self.issensecontinuation.add('_'.join(splited_sense[:i]))

            if sense_len not in self.senses:
                self.senses[sense_len] = set()

            self.senses[sense_len].add(sense)
    
    @staticmethod
    def _filter_tokens(tokens):
        return [token for token in tokens if token.token.isalpha() and not token.token.isupper()]
    
    @staticmethod
    def _sanity_check(tokens, text):
        for token in tokens:
            if token.word != text[token.shift:token.shift + token.word_len]:
                raise Exception(f'ERROR: {token.word} != {text[token.shift:token.shift + token.word_len]}')
                
    def detect(self, text):
        tokens = self._filter_tokens(self.text_preprocessor.tokenize_text(text))
        self._sanity_check(tokens, text)
        concepts = []

        start_w_i = 0
        while start_w_i < len(tokens):
            last_ok_tokens = [tokens[start_w_i]]
            for end_w_i in range(start_w_i + 1, len(tokens) + 1):
                if end_w_i - start_w_i > 1 and tokens[end_w_i-1].num - tokens[end_w_i-2].num != 1:
                    break
                word = '_'.join([t.token for t in tokens[start_w_i:end_w_i]])
                if word in self.senses[end_w_i - start_w_i]:
                    if end_w_i - start_w_i > 1:
                        last_ok_tokens = tokens[start_w_i:end_w_i]

                if word not in self.issensecontinuation:
                    break
            if '_'.join([t.token for t in last_ok_tokens]) in self.senses[len(last_ok_tokens)]:
                concepts.append(last_ok_tokens)
            start_w_i += len(last_ok_tokens)
        
        concepts = [ConceptToken(concept_tokens, self.wordnet.sense2synid['_'.join([t.token for t in concept_tokens])]) for concept_tokens in concepts]
        return concepts
    
class RandomConceptsSubsetSelector:
    def __init__(self):
        pass
    
    def select(self, concepts, max_concepts_count=5, only_deterministic=False):
        if only_deterministic:
            concepts = [c for c in concepts if len(c.synsets_ids) == 1]
        concept_count = min(max_concepts_count, len(concepts))
        if concept_count == 0:
            return []
        
        idx = np.random.choice(range(len(concepts)), concept_count, replace=False)
        selected_concepts = np.array(concepts)[idx].tolist()
        return sorted(selected_concepts, key=lambda x: x.tokens[0].num)
    
class RandomConceptSelector:
    def __init__(self, wordnet):
        self.wordnet = wordnet
    
    def select(self, concept):
        sense = concept.get_sense_norm()
        synsets_ids = self.wordnet.sense2synid[sense]
        return np.random.choice(synsets_ids, 1).tolist()[0]
    
class RandomConceptSenseSelector:
    def __init__(self, wordnet):
        self.wordnet = wordnet
        
    def select(self, synset_id):
        senses = list(self.wordnet.synsets[synset_id].synset_words)
        idx = np.random.choice(range(len(senses)), 1).tolist()[0]
        return senses[idx]
    
class Augmentation:
    def __init__(self, text, concepts, senses, augmentation, restored_augmentation='', score =1.0):
        self.text = text
        self.concepts = concepts
        self.senses = senses
        self.augmentation = augmentation
        self.restored_augmentation = restored_augmentation
        self.score = score

class ConceptAugment:
    def __init__(self, wordnet, concept_subset_selector, concept_selector, sense_selector):
        self.wordnet = wordnet
        self.concept_subset_selector = concept_subset_selector
        self.concept_selector = concept_selector
        self.sense_selector = sense_selector
        
    def augment_one(self, concepts, text, max_concepts_count, only_deterministic=False):
        concepts_subset = self.concept_subset_selector.select(
            copy.deepcopy(concepts), max_concepts_count=max_concepts_count, 
            only_deterministic=only_deterministic
        )
        synset_ids = [self.concept_selector.select(c) for c in concepts_subset]
        senses = [self.sense_selector.select(synid) for synid in synset_ids]
        
        return inject_augmentation(concepts_subset, senses, text), concepts_subset, senses
    
    def augmentate(self, concepts, text, max_concepts_count, augmentations_count, only_deterministic=False):
        augmentations = []
        for i in range(augmentations_count):
            augmentation, concepts_subset, senses = self.augment_one(
                concepts, text, max_concepts_count, only_deterministic
            )
            augmentations.append(Augmentation(text, concepts_subset, senses, augmentation))
            
        return augmentations
    
class T5TextRestore:
    def __init__(self, model_path, device='cuda', prefix='thes_augm: '):
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)
        self.model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
        self.model.eval()
        self.device = device
        self.prefix = prefix
        
    def restore(self, text):
        text = self.prefix + text
        with torch.no_grad():
            model_input = self.tokenizer(text, return_tensors='pt', padding=True).to(self.device)
            max_size = int(model_input.input_ids.shape[1] * 1.5 + 10)
            out = self.model.generate(**model_input, max_length=max_size).detach().cpu()[0]
            result = self.tokenizer.decode(out, skip_special_tokens=True)
        return result
    
    def restore_batch(self, texts, bs=4):
        texts = [self.prefix + t for t in texts]
        batch_count = len(texts) // bs + int(len(texts) % bs != 0)
        results = []
        for i in tqdm(range(batch_count)):
            batch = texts[i * bs: (i + 1) * bs]
            model_input = self.tokenizer(batch, return_tensors='pt', padding=True).to(self.device)
            max_size = int(model_input.input_ids.shape[1] * 1.5 + 10)
            out = self.model.generate(**model_input, max_length=max_size).detach().cpu()
            result = [self.tokenizer.decode(o, skip_special_tokens=True) for o in out]
            results += result
        return results
        
class GPT2Score:
    def __init__(self, model_path, device='cuda'):
        self.model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        self.device = device
        
    def score(self, text):
        max_length = self.model.config.n_positions
        stride = 512
        encodings = self.tokenizer(text, return_tensors='pt')
        nlls = []
        for i in range(0, encodings.input_ids.size(1), stride):
            begin_loc = max(i + stride - max_length, 0)
            end_loc = min(i + stride, encodings.input_ids.size(1))
            trg_len = end_loc - i    # may be different from stride on last loop
            input_ids = encodings.input_ids[:,begin_loc:end_loc].to(self.device)
            target_ids = input_ids.clone()
            target_ids[:,:-trg_len] = -100

            with torch.no_grad():
                outputs = self.model(input_ids, labels=target_ids)
                neg_log_likelihood = outputs[0] * trg_len

            nlls.append(neg_log_likelihood)

        ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
        return float(ppl.detach().cpu())
    
class WordnetAugmentator:
    def __init__(self, wordnet, t5_model_path, gpt2_model_path):
        self.wordnet = wordnet
        self.text_preprocessor = TextPreprocessor()
        self.concept_detector = ConceptDetector(wordnet, self.text_preprocessor)
        
        concepts_subset_selector = RandomConceptsSubsetSelector()
        concept_selector = RandomConceptSelector(wordnet)
        sense_selector = RandomConceptSenseSelector(wordnet)
        self.concept_augmentator = ConceptAugment(wordnet, concepts_subset_selector, concept_selector, sense_selector)
        
        self.t5_text_resorer = T5TextRestore(t5_model_path)
        self.gpt2_score = GPT2Score(gpt2_model_path)
        
    def augmentate(self, text, augmentations_count, topk, max_concept_count=5, bs=1):
        detected_concepts = self.concept_detector.detect(text)
        augmentations = self.concept_augmentator.augmentate(
            detected_concepts, text, max_concept_count, augmentations_count
        )
        
        if bs > 1:
            restored_augmentations = self.t5_text_resorer.restore_batch(
                [augm.augmentation for augm in tqdm(augmentations)], bs=bs
            )
        else:
            restored_augmentations = []
            for augm in tqdm(augmentations):
                restored_augmentations.append(self.t5_text_resorer.restore(augm.augmentation))
                
        for i in range(len(restored_augmentations)):
            augmentations[i].restored_augmentation = restored_augmentations[i]
            
        for augm in tqdm(augmentations):
            augm.score = self.gpt2_score.score(augm.restored_augmentation)
            
        augmentations = sorted(augmentations, key=lambda x: x.score)
        augmentations = self._filter_duplicates(augmentations)
        return augmentations[:topk]

    @staticmethod
    def _filter_duplicates(augmentations):
        processed = set()
        filtered_augmentations = []
        for augm in augmentations:
            if augm.text == augm.restored_augmentation:
                continue
            if augm.restored_augmentation in processed:
                continue
            filtered_augmentations.append(augm)
            processed.add(augm.restored_augmentation)
            
        return filtered_augmentations
    
#====================================
def inject_concept_tokens(concept_tokens, text):
    shift = 0
    text_injected = ''
    for concept_token in concept_tokens:
        first_token = concept_token.tokens[0]
        last_token = concept_token.tokens[-1]

        info = str(concept_token)
        text_injected += text[shift:first_token.shift] + info

        shift = last_token.shift + last_token.word_len

    text_injected += text[shift:]
    return text_injected
    
def inject_augmentation(concept_tokens, senses, text):
    text_augmented = ''
    shift = 0

    for concept, sense in zip(*[concept_tokens, senses]):
        sense = sense.replace('_', ' ')

        first_token = concept.tokens[0]
        last_token = concept.tokens[-1]

        text_augmented += text[shift:first_token.shift] + f'{sense}'
        shift = last_token.shift + last_token.word_len

    text_augmented += text[shift:]
    return text_augmented

In [ ]:
t5_model_path = r'D:\WorkFolder\data\t5_augm\rut5_base_restorer_1m'
gpt2_model_path = 'sberbank-ai/rugpt3small_based_on_gpt2'
wordnet_augmentator = WordnetAugmentator(wordnet, t5_model_path, gpt2_model_path)


In [ ]:
%%time
text = 'Ее выполнение позволит правительственным войскам САР установить контроль над сирийско-турецкой границей.'
augmentations = wordnet_augmentator.augmentate(text, 50, 5, bs=10)
for augm in augmentations:
    print(augm.restored_augmentation)
    print(augm.score)

In [ ]:
import pandas as pd

test = pd.read_csv(r'D:\WorkFolder\data\augmentation_data_news2017\0_sentences_test.csv')
test['text'] = test['text'].apply(lambda x: x.replace('\xa0', ' '))

In [ ]:
for text in test['text']:
    augmentations = wordnet_augmentator.augmentate(text, 50, 5, bs=10)
    for augm in augmentations:
        print(augm.restored_augmentation)
        print(augm.score)

In [ ]:
text

In [ ]:
texts = test.sample(4)['corrupted_text'].tolist()
texts

In [ ]:
restore_batch(
    texts, 
    wordnet_augmentator.t5_text_resorer.model, 
    wordnet_augmentator.t5_text_resorer.tokenizer, 
    wordnet_augmentator.t5_text_resorer.prefix, 
    wordnet_augmentator.t5_text_resorer.device,
    max_bs = 3
)

In [ ]:
texts

In [ ]:
text = 'Ее выполнение позволит правительственным войскам САР установить контроль над сирийско-турецкой границей.'
for res in wordnet_augmentator.augmentate(text, 50, 5):
    print(res)

In [ ]:
print(text)
print('---'*10)
print(inject_concept_tokens(detected_concepts, text))
print('---'*10)
for i in range(5):
    augmented_text = augmentator.augment_one(detected_concepts, text)
    restored_augmented_text = t5_text_resorer.restore(augmented_text)
    score = gpt2_score.score(restored_augmented_text)
    print(augmented_text)
    print(restored_augmented_text)
    print(score)
    print('---'*10)

In [ ]:
wordnet.synsets['673-N'].synset_words

In [ ]:
detected_concepts

In [ ]:
inject_tokens(detected_concepts, text)

In [ ]:
wordnet.sense2synid['турецкий']

In [ ]:
word = 'Словестная'
%timeit text_preprocessor.morph_analyzer.parse(word)[0].normal_form

In [ ]:
word = 'многие-многие'
text_preprocessor.get_morph(word)

In [ ]:
word.isalpha()

In [ ]:
for a, b in zip(*[[1], [2]]):
    print(a)
    print(b)

In [ ]:
from socket import gethostbyaddr, getfqdn, gethostname

In [ ]:
help(gethostbyaddr)

In [ ]:
gethostbyaddr('127.0.0.0')

In [ ]:
gethostname()

In [ ]:
text = 'В Совфеде {{обсудили|обсудить|106800-V}} {{важнейший|важный|119261-A,112920-A}} {{проект|проект|106466-N,7059-N,112123-N,138540-N}} по {{освоению|освоение|130167-N,106685-N}} {{российской|российский|2636-A}} {{Арктики|арктика|105540-N}}.'
text = text.split()


In [ ]:
import re

re.findall(r'{{(.*?)}}', text[3])[0].split('|')